In [46]:
import pandas as pd
import numpy as np
import animalhelper as ah

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [247]:
df = pd.read_csv('./data/engineered.csv', index_col = 0, dtype={'intake_condition': str})

In [248]:
df = df[~(df['group'] == 'Unknown')]

## Generate df_mod

In [258]:
df.columns.values

array(['age', 'animal_id', 'animal_type', 'datetime_in',
       'days_in_shelter', 'dow', 'fixed_status', 'gender', 'group',
       'intake_condition', 'intake_season', 'intake_type', 'mix', 'name',
       'outcome_type', 'simple_color'], dtype=object)

In [264]:
df['datetime_in'] = pd.to_datetime(df['datetime_in'])

In [265]:
df['hour_in'] = df['datetime_in'].dt.hour

In [266]:
df['hour_in'] = df['hour_in'].astype(str)

In [226]:
df_hour = pd.concat([df['animal_id'], pd.get_dummies(df['hour_in'])], axis=1)

In [268]:
df_mod = pd.concat([df[['animal_id', 'age']],
           pd.get_dummies(df[['animal_type', 'dow', 'fixed_status', 'gender', 'group',
                             'intake_condition', 'intake_season', 'intake_type', 'mix', 'name', 
                             'simple_color', 'hour_in']]), df[['outcome_type', 'days_in_shelter']]], axis=1)

In [227]:
df_mod = pd.DataFrame()

## Impute unknown fixed status values

In [250]:
df['fixed_status'] = df['fixed_status'].apply(lambda x: ah.impute_fixed_status(x))

# Clean up groups

In [235]:
df['group'].value_counts()

short hair       27058
Terrier           9524
Toy               7376
Sporting          6530
Herding           5897
Working           4604
Hound             2995
Non-Sporting      1870
long hair         1245
Terrier & Toy      111
Unknown              1
Name: group, dtype: int64

In [236]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67211 entries, 0 to 28302
Data columns (total 17 columns):
age                 67211 non-null int64
animal_id           67211 non-null object
animal_type         67211 non-null object
datetime_in         67211 non-null datetime64[ns]
days_in_shelter     67211 non-null int64
dow                 67211 non-null int64
fixed_status        67211 non-null object
gender              67211 non-null int64
group               67211 non-null object
intake_condition    67211 non-null object
intake_season       67211 non-null object
intake_type         67211 non-null object
mix                 67211 non-null int64
name                67211 non-null int64
outcome_type        67211 non-null object
simple_color        67211 non-null object
hour_in             67211 non-null object
dtypes: datetime64[ns](1), int64(6), object(10)
memory usage: 9.2+ MB


In [237]:
def clean_groups(s):
    if s == 'Terrier & Toy':
        return 'Terrier'
    return s

In [251]:
df['group'] = df['group'].apply(lambda x: clean_groups(x))

In [254]:
df.reset_index(inplace=True, drop=True)

## Generate dummy variable dataframe

In [269]:
df_mod = pd.concat([df[['animal_id', 'age']],
           pd.get_dummies(df[['animal_type', 'dow', 'fixed_status', 'gender', 'group',
                             'intake_condition', 'intake_season', 'intake_type', 'mix', 'name', 
                             'simple_color', 'hour_in']]), df[['outcome_type', 'days_in_shelter']]], axis=1)

In [271]:
df_mod.head()

,animal_id,age,dow,gender,mix,name,animal_type_Cat,animal_type_Dog,fixed_status_fixed,fixed_status_intact,...,hour_in_23,hour_in_3,hour_in_4,hour_in_5,hour_in_6,hour_in_7,hour_in_8,hour_in_9,outcome_type,days_in_shelter
0,A706918,2920,6,1,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,Return to Owner,0
1,A724273,330,3,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,Return to Owner,6
2,A682524,1460,6,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,Return to Owner,3
3,A743852,730,5,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,Return to Owner,3
4,A708452,730,3,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,Return to Owner,5


In [272]:
df_mod.to_csv('./data/all_categoricals.csv')

In [273]:
df.to_csv('./data/master_df_1129.csv')